# IMPORT LIBRARIES

In [24]:
import os
import glob
import ROOT
from math import sqrt
import time
#from os import path
from pathlib import Path
import re
import copy
from tqdm.notebook import trange, tqdm
import numpy as np
from array import *
%jsroot on

# LOAD SPLINES AND DEFINE BINNING

In [25]:
35.9 + 41.5 + 59.7

137.10000000000002

In [26]:
lumi = {'MC_2016_CorrectBTag':35.9, 'MC_2017':41.5, 'MC_2018':59.7}

In [27]:
D2jetZHSpline = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DjjZH_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
D2jetWHSpline = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DjjWH_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
D2jetVBFSpline = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DjjVBF_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

#WPCshift = 1
WPCshift2jv = 0.46386/(1. - 0.46386)
WPCshift2jz = 0.91315/(1. - 0.91315)
WPCshift2jw = 0.88384/(1. - 0.88384)

In [28]:
DbkgkinSpline4e = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dbkgkin_4e_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgkinSpline4mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dbkgkin_4mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgkinSpline2e2mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dbkgkin_2e2mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

DggbkgkinSpline4e = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dggbkgkin_4e_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DggbkgkinSpline4mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dggbkgkin_4mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DggbkgkinSpline2e2mu = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_Dggbkgkin_2e2mu_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

In [29]:
DbkgjjEWQCDSpline4lHadVH = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_4l_HadVHTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgjjEWQCDSpline2l2lHadVH = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_2l2l_HadVHTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

DbkgjjEWQCDSpline4lJJVBF = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_4l_JJVBFTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")
DbkgjjEWQCDSpline2l2lJJVBF = ROOT.TFile("/eos/user/l/lkang/Active_Research/Discriminants/RecoMEConstants/SmoothKDConstant_m4l_DbkgjjEWQCD_2l2l_JJVBFTagged_13TeV.root").Get("sp_gr_varReco_Constant_Smooth")

In [30]:
medges = np.array([220, 230, 240, 250, 260, 280, 310, 340, 370, 400, 475, 550, 625, 700, 800, 900, 1000, 1200, 1600, 2000, 3000, 13000], dtype='float64')
# medges = np.array([220, 230, 240, 250, 260, 280, 310, 340, 370, 400, 475, 550, 625, 700, 800, 900, 1000, 1200, 1600, 2000, 13000], dtype='float64')
## medges = np.array([220, 260, 370, 625, 1000, 13000], dtype='float64')
## medges = np.array([220, 400, 3000, 13000], dtype='float64')
# medges = 5*np.arange(401, dtype='float64')
#d1edges = np.arange(21, dtype='float64') / 20 * 12780 + 220
##d1edges = np.array([ 220.,  316.,  412.,  508.,  604.,  700.,  796.,  892.,  988., 1084., 1180., 1276., 1372., 1468., 1564., 1660., 1756., 1852., 1948., 2044., 13000])
##d1edges = np.array([ 220.,  828., 1436., 2044.])
#d1edges = np.array([220, 4480, 8740, 13000], dtype='float64')
#d1edges = np.array([220, 13000], dtype='float64')
d1edges = np.arange(21, dtype='float64') / 20
#d1edges = np.arange(6, dtype='float64') / 5
# d1edges = np.arange(4, dtype='float64') / 3
# d1edges = np.arange(3, dtype='float64') / 2
d2edges = np.arange(21, dtype='float64') / 10 - 1
#d2edges = np.arange(6, dtype='float64') / 2.5 - 1
# d2edges = np.arange(4, dtype='float64') / 1.5 - 1
# d2edges = np.arange(3, dtype='float64') / 1 - 1

In [31]:
print("medges", len(medges))
print("medges", (medges))
print("d1edges", len(d1edges))
print("d1edges", (d1edges))
print("d2edges", len(d2edges))
print("d2edges", (d2edges))

medges 22
medges [  220.   230.   240.   250.   260.   280.   310.   340.   370.   400.
   475.   550.   625.   700.   800.   900.  1000.  1200.  1600.  2000.
  3000. 13000.]
d1edges 21
d1edges [0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]
d2edges 21
d2edges [-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


# CHOOSE INPUT TREES

In [9]:
#tree i made  "/eos/user/s/skyriaco/Offshell/200205_CutBased/"

### Read selected trees from user-specified file

In [32]:
# treelistpath = "/eos/user/l/lkang/Active_Research/Discriminants/alltaggedtrees.txt"
#treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/outputtree_list.txt"
#treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/alloutput.txt"
#treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/output_highmass1618.txt"
treelistpath = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/output_highmass18.txt"
treelist = []

with open(treelistpath) as f:
    llist = [line.rstrip() for line in f]
        
for line in llist:
    if Path(line).exists(): treelist.append(line)

In [33]:
for t in treelist:
    #if "VBF" in t:
         print(t)

/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/ExtraIgnore/DYToLL_2J/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1000/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH115/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH135/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH140/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH145/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH150/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1500/ZZ4lAnalysis.root
/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH155/ZZ4lAnalysis.root
/eo

In [34]:
yeardict = {}

for numfile in trange(len(treelist)):
    filename = treelist[numfile]
    ind = filename.split("/").index("200205_CutBased")
#     tagtreefile = "/".join(filename.split("/")[ind:])
    year = filename.split("/")[ind:][1]
#     print(year)
    if year not in yeardict.keys():
        yeardict[year] = {}
    else:
        if "ZZTo4l" in filename.split("/")[ind:][2]: prod = "ZZTo4l"
        else: prod = filename.split("/")[ind:][3]
            
        if prod not in yeardict[year]:
        #    if 'gg' in prod:
        #        yeardict[year][prod] = [[], [], []]
        #    else:
                yeardict[year][prod] = [[]]
        if 'gg' in prod:
            yeardict[year][prod][0].append(filename)
            #if 'ggTo2e2mu' in filename:
            #    yeardict[year][prod][0].append(filename)
            #elif 'ggTo4e' in filename:
            #    yeardict[year][prod][1].append(filename)
            #elif 'ggTo4mu' in filename:
            #    yeardict[year][prod][2].append(filename)
        elif "VBF" in prod:
            yeardict[year][prod][0].append(filename)
        elif "ZH" in prod:    
            yeardict[year][prod][0].append(filename)
        elif "Wplus"   in prod:    
            yeardict[year][prod][0].append(filename)  
        elif "Wminus"   in prod:    
            yeardict[year][prod][0].append(filename)   
        elif prod == "ZZTo4l":
            yeardict[year][prod][0].append(filename)
        else:
            print("ERROR: Cannot recognize production mode of " + filename + "! Tree not sorted!")

  0%|          | 0/155 [00:00<?, ?it/s]

### Check organized trees

In [35]:
yeardict

{'MC_2018': {'VBFH1000': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1000/ZZ4lAnalysis.root']],
  'VBFH115': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH115/ZZ4lAnalysis.root']],
  'VBFH135': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH135/ZZ4lAnalysis.root']],
  'VBFH140': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH140/ZZ4lAnalysis.root']],
  'VBFH145': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH145/ZZ4lAnalysis.root']],
  'VBFH150': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH150/ZZ4lAnalysis.root']],
  'VBFH1500': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1500/ZZ4lAnalysis.root']],
  'VBFH155': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/

In [36]:
print (yeardict)

{'MC_2018': {'VBFH1000': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1000/ZZ4lAnalysis.root']], 'VBFH115': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH115/ZZ4lAnalysis.root']], 'VBFH135': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH135/ZZ4lAnalysis.root']], 'VBFH140': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH140/ZZ4lAnalysis.root']], 'VBFH145': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH145/ZZ4lAnalysis.root']], 'VBFH150': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH150/ZZ4lAnalysis.root']], 'VBFH1500': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMass/VBFH1500/ZZ4lAnalysis.root']], 'VBFH155': [['/eos/user/s/skyriaco/Offshell/Syst/highmass/MC_18/200205_CutBased/MC_2018/HighMa

### Account for all bespoke EW MELA branches

In [37]:
refsam4l = ["VBFToContinToZZTo4l", "VBFToHiggs0L1ContinToZZTo4l", "VBFToHiggs0L1f025ph0ToZZTo4l", "VBFToHiggs0L1f05ph0ContinToZZTo4l", "VBFToHiggs0L1f05ph0ToZZTo4l", "VBFToHiggs0L1f075ph0ToZZTo4l", "VBFToHiggs0L1ToZZTo4l", "VBFToHiggs0MContinToZZTo4l", "VBFToHiggs0Mf025ph0ToZZTo4l", "VBFToHiggs0Mf05ph0ContinToZZTo4l", "VBFToHiggs0Mf05ph0ToZZTo4l", "VBFToHiggs0Mf075ph0ToZZTo4l", "VBFToHiggs0MToZZTo4l", "VBFToHiggs0PHContinToZZTo4l", "VBFToHiggs0PHf025ph0ToZZTo4l", "VBFToHiggs0PHf05ph0ContinToZZTo4l", "VBFToHiggs0PHf05ph0ToZZTo4l", "VBFToHiggs0PHf075ph0ToZZTo4l", "VBFToHiggs0PHToZZTo4l", "VBFToHiggs0PMContinToZZTo4l", "VBFToHiggs0PMToZZTo4l"]

In [38]:
refmel = ["p_Gen_JJEW_BKG_MCFM", "p_Gen_JJEW_BSI_ghv1_0_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1177p11_MCFM", "p_Gen_JJEW_BSI_ghv1_1_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m2038p82_MCFM", "p_Gen_JJEW_SIG_ghv1_0_ghv1prime2_m1549p165_MCFM", "p_Gen_JJEW_BSI_ghv1_0_ghv4_0p216499_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv4_0p164504_MCFM", "p_Gen_JJEW_BSI_ghv1_1_ghv4_0p216499_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv4_0p216499_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv4_0p284929_MCFM", "p_Gen_JJEW_SIG_ghv1_0_ghv4_0p216499_MCFM", "p_Gen_JJEW_BSI_ghv1_0_ghv2_0p207049_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv2_0p157323_MCFM", "p_Gen_JJEW_BSI_ghv1_1_ghv2_0p207049_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv2_0p207049_MCFM", "p_Gen_JJEW_SIG_ghv1_1_ghv2_0p272492_MCFM", "p_Gen_JJEW_SIG_ghv1_0_ghv2_0p207049_MCFM", "p_Gen_JJEW_BSI_ghv1_1_MCFM", "p_Gen_JJEW_SIG_ghv1_1_MCFM"]

In [40]:
refden = dict(zip(refsam4l, refmel))

In [41]:
refden

{'VBFToContinToZZTo4l': 'p_Gen_JJEW_BKG_MCFM',
 'VBFToHiggs0L1ContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_0_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0L1f025ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1177p11_MCFM',
 'VBFToHiggs0L1f05ph0ContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_1_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0L1f05ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0L1f075ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv1prime2_m2038p82_MCFM',
 'VBFToHiggs0L1ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_0_ghv1prime2_m1549p165_MCFM',
 'VBFToHiggs0MContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_0_ghv4_0p216499_MCFM',
 'VBFToHiggs0Mf025ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv4_0p164504_MCFM',
 'VBFToHiggs0Mf05ph0ContinToZZTo4l': 'p_Gen_JJEW_BSI_ghv1_1_ghv4_0p216499_MCFM',
 'VBFToHiggs0Mf05ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv4_0p216499_MCFM',
 'VBFToHiggs0Mf075ph0ToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_1_ghv4_0p284929_MCFM',
 'VBFToHiggs0MToZZTo4l': 'p_Gen_JJEW_SIG_ghv1_0_ghv4_0p216499_MCFM',
 'V

# REPEAT THIS SECTION AS NEEDED TO CREATE TEMPLATES AND FILL 'hlist'

##### Note that 'hlist' is expected to look like ['ggH SIG', 'ggH BSI', 'ggH BKG', 'EW SIG', 'EW BSI', 'EW BKG', 'qqbar BKG'] so make the appropriate histograms then append to 'hlist'
##### As you change the year, production mode, final state, SIG/BKG/INT mix, and category, you should re-run the 'Fill histogram' cell and check the output before adding the next template to our 'hlist'

### Set up basic information. Make list to store histograms---do not evaluate this cell unless you want an empty instance of hlist!

In [42]:
hlist = ['ggH SIG', 'ggH BSI', 'ggH BKG', 'qqbar BKG']

In [43]:
targetreweight = {"gg": ["p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal", "p_Gen_GG_BSI_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal", "p_Gen_GG_BKG_MCFM*KFactor_QCD_ggZZ_Nominal"],
                 "VBF": ["p_Gen_JJEW_SIG_ghv1_1_MCFM*0.5", "p_Gen_JJEW_BSI_ghv1_1_MCFM*0.5", "p_Gen_JJEW_BKG_MCFM*0.5"],
                 "ZZTo4l": ["1"]}

finalstate = {"4e": "121*121", "4mu": "169*169", "2e2mu": "121*169"}

In [44]:
proc = ["ggH_0PM", "ggH_g11g21_negative", "ggH_g11g21_positive", "back_ggZZ", "back_qqZZ", "data_obs"]

In [22]:
#"qqH_0PM", "qqH_g12g22_negative", "qqH_g12g22_positive", "back_VVZZ"

In [45]:
print year

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(year)? (2462559766.py, line 1)

### Choose year, production mode, final state, SIG/BKG/INT mix, and category

In [46]:
#configure what you would like to run

ltargetyear = ["MC_2018"] #,"MC_2016_CorrectBTag","MC_2018"]
ltargetprod = ["gg"] #,"VBF","ZZTo4l"]
ltargetstate = ["4l"] #"4mu","4e","2e2mu"]
ltargetcomp = [0,1,2]    # SIG=0 BSI=1 BKG=2    (for the qqbar bkg events in 'ZZTo4l' this choice does not matter as you can see above)
ltargetcateg = [0,1,2]  # Untag=0 VBF=1 VH=2

### Fill histogram

In [47]:
#syst_weights = []



def FillHist(targetprod,targetcomp,targetcateg,h_list) :

    print (targetreweight[targetprod][targetcomp])
    print (targetprod,targetcomp,targetcateg)
    weight = "{}*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp])
    weightewzzup = "{}*(1.0 + KFactor_EW_qqZZ_unc)*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp])
    weightewzzdn = "{}*(1.0 - KFactor_EW_qqZZ_unc)*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp])
    
    weightqrup = "{}*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp])
    weightqrdn = "{}*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40".format(targetreweight[targetprod][targetcomp])

    
    LHEweightsUp = ["LHEweight_QCDscale_muR2_muF1","LHEweight_QCDscale_muR1_muF2","LHEweight_PDFVariation_Up","LHEweight_AsMZ_Up","PythiaWeight_isr_muR4 * PythiaWeight_fsr_muR4"]
    LHEweightsDn = ["LHEweight_QCDscale_muR0p5_muF1","LHEweight_QCDscale_muR1_muF0p5","LHEweight_PDFVariation_Dn","LHEweight_AsMZ_Dn","PythiaWeight_isr_muR0p25 * PythiaWeight_fsr_muR0p25"]
    
    
    hf_LHEup = []
    hf_LHEdn = []
    
    for i,w_ght in enumerate(LHEweightsUp) : 
          LHEweightsUp[i] = w_ght+"*"+weight
          print (LHEweightsUp[i])  
          LHEweightsDn[i] = LHEweightsDn[i]+"*"+weight
          h_temp = ROOT.TH1F("hf_temp","",len(medges)-1, medges)
          h_tempp = copy.deepcopy(h_temp)  
          hf_LHEup.append(h_tempp)
          h_temppdn = copy.deepcopy(h_temp)  

          hf_LHEdn.append(h_temppdn)
 
   
    hf = ROOT.TH3F("hf","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
    hf_systdn = ROOT.TH1F("hf_kew_systdn","",100,0,2000)
    hf_systup = ROOT.TH1F("hf_kew_systup","",100,0,2000)
    hf_nom    = ROOT.TH1F("hf_nom","",100,0,2000)
    hf_nomlhe = ROOT.TH1F("hf_nomlhe","",len(medges)-1, medges)    
    hf_qr_up  = ROOT.TH1F("hf_qr_up","",100,0,2000)
    hf_qr_dn  = ROOT.TH1F("hf_qr_dn","",100,0,2000)    
    
    
    hf.SetDirectory(0)
    hf_systdn.SetDirectory(0)
    hf_systup.SetDirectory(0)
    hf_nom.SetDirectory(0)
    hf_nomlhe.SetDirectory(0)
    for i,ht in enumerate(LHEweightsUp) :              
            hf_LHEup[i].SetDirectory(0)
            hf_LHEdn[i].SetDirectory(0)

    for keynum in trange(len(yeardict.keys())):
        year = list(yeardict.keys())[keynum]
        #if year != targetyear: continue
        #print("\n", year, lumi[year])

        hs = ROOT.TH3F("hs","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
        hs_systdn = ROOT.TH1F("hs_kew_systdn","",100,0,2000)
        hs_systup = ROOT.TH1F("hs_kew_systup","",100,0,2000)
        hs_nom    = ROOT.TH1F("hs_nom","",100,0,2000)
        hs_nomlhe = ROOT.TH1F("hs_nomlhe","",len(medges)-1, medges)
        hs_LHEup = []
        hs_LHEdn = []                
        for i,ht in enumerate(hf_LHEup) :   
            hs_temp = ROOT.TH1F("hs_temp","",len(medges)-1, medges)
            hs_temp.SetDirectory(0)
            hs_tempp = copy.deepcopy(hs_temp) 
            hs_LHEup.append(hs_tempp)
            
            hs_temppdn = copy.deepcopy(hs_temp) 
            hs_LHEdn.append(hs_temppdn)
        
        
        hs_qr_up  = ROOT.TH1F("hs_qr_up","",100,0,2000)
        hs_qr_dn  = ROOT.TH1F("hs_qr_dn","",100,0,2000)        
        hs.SetDirectory(0)
        hs_systdn.SetDirectory(0)
        hs_systup.SetDirectory(0)
        hs_nom.SetDirectory(0)
        hs_nomlhe.SetDirectory(0)
 
            
        for prodd in yeardict[year]: 
          if targetprod in prodd :   
           for decay in yeardict[year][prodd]:
            #         if "2e2mu" not in decay[0]: continue
            #print("\n", decay)
            ht = ROOT.TH3F("ht","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
            ht_systdn = ROOT.TH1F("ht_kew_systdn","",100,0,2000)
            ht_systup = ROOT.TH1F("ht_kew_systup","",100,0,2000)
            ht_nom    = ROOT.TH1F("ht_nom","",100,0,2000)
            ht_nomlhe = ROOT.TH1F("ht_nomlhe","",len(medges)-1, medges)
            
        
            ht_qr_up  = ROOT.TH1F("ht_qr_up","",100,0,2000)
            ht_qr_dn  = ROOT.TH1F("ht_qr_dn","",100,0,2000)
            
            ht_LHEup = []
            ht_LHEdn = []        
            for i,htf in enumerate(hf_LHEup) :   
                ht_temp = ROOT.TH1F("ht_temp","",len(medges)-1, medges)
                ht_temp.SetDirectory(0)
                ht_tempp = copy.deepcopy(ht_temp) 
                ht_LHEup.append(ht_tempp)
                ht_tempdn = ROOT.TH1F("ht_tempdn","",len(medges)-1, medges) 
                ht_temppdn  = copy.deepcopy(ht_tempdn)
                ht_LHEdn.append(ht_temppdn)
                
                
            ht.SetDirectory(0)
            ht_systdn.SetDirectory(0)
            ht_systup.SetDirectory(0)
            ht_nom.SetDirectory(0)
            ht_nomlhe.SetDirectory(0)
            ht_qr_up.SetDirectory(0)  
            ht_qr_dn.SetDirectory(0)  
            for i,htff in enumerate(ht_LHEup) :   
            
                ht_LHEup[i].SetDirectory(0)
                ht_LHEdn[i].SetDirectory(0)
            count = 0
            
            
            for tfile in trange(len(decay)):

                #print(decay[tfile])
                if "VBF" in decay[tfile]:

                    #if "0L1" in decay[tfile]: continue
                    skey = decay[tfile].split("/")[-2].replace('_M125_GaSM', '')
                    sampleweight = refden[skey]
    #                 print(sampleweight)
                if ( os.stat(decay[tfile]).st_size > 100 ):  
                        f = ROOT.TFile(decay[tfile])
                        t = f.Get("eventTree")
                        htt = ROOT.TH3F("htt","", len(medges)-1, medges, len(d1edges)-1, d1edges, len(d2edges)-1, d2edges)
                        htt_ew_systdn = ROOT.TH1F("htt_ew_systdn","",100,0,2000)
                        htt_ew_systup = ROOT.TH1F("htt_ew_systup","",100,0,2000)
                        htt_nom    = ROOT.TH1F("htt_nom","",100,0,2000)
                        htt_nomlhe = ROOT.TH1F("htt_nomlhe","",len(medges)-1, medges)

                        htt_LHEup = []
                        htt_LHEdn = []        
                
                        for i,htff in enumerate(ht_LHEup) :
                            #print (LHEweightsUp[i])
                            htt_tempu = ROOT.TH1F("htt_tempu","",len(medges)-1, medges)
                            t.Draw("ZZMass>>htt_tempu",LHEweightsUp[i],"")                                
                            htt_LHEup.append(htt_tempu)
                            #print (LHEweightsDn[i])
                            #print ("----------------------")
                            htt_tempd = ROOT.TH1F("htt_tempd","",len(medges)-1, medges)
                            t.Draw("ZZMass>>htt_tempd",LHEweightsDn[i],"")
                            htt_LHEdn.append(htt_tempd)
                            #print ("up:",htt_tempu.Integral(),htt_LHEup[i].Integral())
                            #print ("dn:",htt_tempd.Integral(),htt_LHEdn[i].Integral())
                         

                        htt_qr_up  = ROOT.TH1F("htt_qr_up","",100,0,2000)
                        htt_qr_dn  = ROOT.TH1F("htt_qr_dn","",100,0,2000)
                        #print (weight_ewzz_up)
                        #print (weight_ewzz_dn)
                        #print (weight)

                        t.Draw("Dbsi:Dbkg:ZZMass>>htt",weight,"")                        
                        t.Draw("ZZMass>>htt_nom",weight,"")
                        t.Draw("ZZMass>>htt_nomlhe",weight,"")
                        #print ("om nomnomm:",htt_nomlhe.Integral())
                        t.Draw("ZZMass>>htt_ew_systup",weightewzzup,"")
                        t.Draw("ZZMass>>htt_ew_systdn",weightewzzdn,"")

                        
                        t.Draw("ZZMass>>htt_qr_up",weightqrup,"")    
                        t.Draw("ZZMass>>htt_qr_dn",weightqrdn,"")    


                        #print ("htt_up:", htt_ew_systup.Integral())
                        #print ("htt_dn:", htt_ew_systdn.Integral())
                        
                        if count == 0:
                            #print("ADDED --- htt = ", htt.Integral())
                            ht.Add(htt)
                            ht_nom.Add(htt_nom)
                            
                            #print (htt_nomlhe)
                            #print (ht_nomlhe)
                            ht_nomlhe.Add(htt_nomlhe)

                            ht_systdn.Add(htt_ew_systdn)
                            ht_systup.Add(htt_ew_systup)
                            
                            ht_qr_up.Add(htt_qr_up)
                            ht_qr_dn.Add(htt_qr_dn)
                            
                            for i,htff in enumerate(ht_LHEup) :   
                                print  ("check addition:",ht_LHEup[i].Integral(), htt_LHEup[i].Integral() + ht_LHEup[i].Integral() )  
                                ht_LHEup[i].Add(htt_LHEup[i])                           
                                print ("hereu",i,ht_LHEup[i].Integral(),htt_LHEup[i].Integral())
                                
                            
                                print  ("check addition:",ht_LHEdn[i].Integral(), htt_LHEdn[i].Integral() + ht_LHEdn[i].Integral() )  
                                ht_LHEdn[i].Add(htt_LHEdn[i])
                                print ("heredn",i,ht_LHEdn[i].Integral(),htt_LHEdn[i].Integral())

                            count+=1
                        elif htt.Integral() > ((ht.Integral()/count) + 2*sqrt((ht.Integral()/count))): 
                            print("SKIPPED --- htt = ", htt.Integral())
                        elif htt.Integral() < ((ht.Integral()/count) - 2*sqrt((ht.Integral()/count))): 
                            print("SKIPPED --- htt = ", htt.Integral())
                        else:
                            print("ADDED --- htt = ", htt.Integral())
                            ht.Add(htt)
                            ht_nom.Add(htt_nom)
                            ht_nomlhe.Add(htt_nomlhe)

                            ht_systdn.Add(htt_ew_systdn)
                            ht_systup.Add(htt_ew_systup)
                            for i,htff in enumerate(ht_LHEup) : 
                                ht_LHEup[i].SetDirectory(0)
                                ht_LHEup[i].Add(htt_LHEup[i])
                                print ("sfin:",ht_LHEup[i].Integral())

                                
                                ht_LHEdn[i].SetDirectory(0)
                                print ("sfindn:",ht_LHEdn[i].Integral())
                                ht_LHEdn[i].Add(htt_LHEdn[i])
                                            
                            ht_qr_up.Add(htt_qr_up)
                            ht_qr_dn.Add(htt_qr_dn)    
                            
                            count+=1
            #print("ht = ", ht.Integral())
            ht.Scale(1./count)
            ht_nom.Scale(1./count)
            ht_systdn.Scale(1./count)
            ht_systup.Scale(1./count)
            
            for i,htff in enumerate(ht_LHEup) :            
                ht_LHEup[i].Scale(1./count)
                ht_LHEdn[i].Scale(1./count)        
            #print("ht = ", ht.Integral())
            hs.Add(ht)
            hs_systdn.Add(ht_systdn)
            hs_systup.Add(ht_systup)
            hs_nom.Add(ht_nom)
            hs_nomlhe.Add(ht_nomlhe)

            hs_qr_up.Add(ht_qr_up)
            hs_qr_dn.Add(ht_qr_dn) 
            
            for i,htff in enumerate(ht_LHEup) :            
                hs_LHEup[i].Add(ht_LHEup[i])
                #print("hs u => ",i, hs_LHEup[i].Integral())
                
                
            for i,htff in enumerate(ht_LHEdn) :            
                hs_LHEdn[i].Add(ht_LHEdn[i])    
                #print("hs d => ",i, hs_LHEdn[i].Integral())


        hf.Add(hs)
        hf_systdn.Add(hs_systdn)
        hf_systup.Add(hs_systup)
        hf_nom.Add(hs_nom)
        hf_nomlhe.Add(hs_nomlhe)
        for i,htff in enumerate(hf_LHEup) : 
            hf_LHEup[i].Add(hs_LHEup[i])
            print ("u",hs_LHEup[i].Integral()) 
                
        
            hf_LHEdn[i].Add(hs_LHEdn[i])
            print ("dn",hs_LHEdn[i].Integral()) 

        hf_qr_up.Add(hs_qr_up)
        hf_qr_dn.Add(hs_qr_dn) 
        print("hf = ", hf_nom.Integral())

        hf.Scale(1/len(yeardict.keys()))
        hf_systdn.Scale(1/len(yeardict.keys()))
        hf_systup.Scale(1/len(yeardict.keys()))
        hf_nom.Scale(1/len(yeardict.keys()))
        
        for i in range(0,len(hf_LHEup)) :            
                hf_LHEup[i].Scale(1/len(yeardict.keys()))
                h_list.append(hf_LHEup[i])
                print ("up:",hf_LHEup[i].Integral())
       
       
                hf_LHEdn[i].Scale(1/len(yeardict.keys()))
                
                h_list.append(hf_LHEdn[i])
                print ("dn:",hf_LHEdn[i].Integral())
                print ("adding to hlist")


        #h_list.append(hf)
        #h_list.append(hf_systdn)
        #h_list.append(hf_systup)
        h_list.append(hf_nomlhe)
        

In [48]:

h_gg_unt =[]
print (h_gg_unt)
FillHist("gg",0,0,h_gg_unt)

#for tcomp in ltargetcomp: 
#       FillHist("gg",tcomp,0,h_gg_unt)
'''
h_gg_vbf =[]
for tcomp in ltargetcomp: 
       FillHist("gg",tcomp,1,h_gg_vbf)

h_gg_vh =[]
for tcomp in ltargetcomp: 
       FillHist("gg",tcomp,2,h_gg_vh)     
        
 
       
#=========================================  
h_ew_unt =[]
for tcomp in ltargetcomp: 
       FillHist("VBF",tcomp,0,h_ew_unt)

h_ew_vbf =[]
for tcomp in ltargetcomp: 
       FillHist("VBF",tcomp,1,h_ew_vbf)

h_ew_vh =[]
for tcomp in ltargetcomp: 
       FillHist("VBF",tcomp,2,h_ew_vh)      
              
#=========================================  
h_bkg_unt =[]
FillHist("ZZTo4l",0,0,h_bkg_unt)

h_bkg_vbf =[]
FillHist("ZZTo4l",0,1,h_bkg_vbf)
    
h_bkg_vh =[]
FillHist("ZZTo4l",0,2,h_bkg_vh)      
'''       

[]
p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal
gg 0 0
LHEweight_QCDscale_muR2_muF1*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_QCDscale_muR1_muF2*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_PDFVariation_Up*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*169)  )*137.1*(ZZMass>=220)*1000*xsec*overallEventWeight*L1prefiringWeight/Bin40
LHEweight_AsMZ_Up*p_Gen_GG_SIG_kappaTopBot_1_ghz1_1_MCFM*KFactor_QCD_ggZZ_Nominal*((Z1Flav*Z2Flav)==(121*121) ||(Z1Flav*Z2Flav)==(169*169) ||(Z1Flav*Z2Flav)==(121*1

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

check addition: 0.0 117.27739881328307
hereu 0 117.27739881328307 117.27739881328307
check addition: 0.0 151.81031124899164
heredn 0 151.81031124899164 151.81031124899164
check addition: 0.0 129.97665035631508
hereu 1 129.97665035631508 129.97665035631508
check addition: 0.0 136.37920367356855
heredn 1 136.37920367356855 136.37920367356855
check addition: 0.0 135.6672691400163
hereu 2 135.6672691400163 135.6672691400163
check addition: 0.0 130.51063387596514
heredn 2 130.51063387596514 130.51063387596514
check addition: 0.0 134.2560419626534
hereu 3 134.2560419626534 134.2560419626534
check addition: 0.0 130.64521700015757
heredn 3 130.64521700015757 130.64521700015757
check addition: 0.0 133.04622332251165
hereu 4 133.04622332251165 133.04622332251165
check addition: 0.0 130.41819686582312
heredn 4 130.41819686582312 130.41819686582312


  0%|          | 0/1 [00:00<?, ?it/s]

...

check addition: 0.0 58.3082495048875
hereu 0 58.3082495048875 58.3082495048875
check addition: 0.0 75.67785026610363
heredn 0 75.67785026610363 75.67785026610363
check addition: 0.0 64.69576325151138
hereu 1 64.69576325151138 64.69576325151138
check addition: 0.0 67.88576095702592
heredn 1 67.88576095702592 67.88576095702592
check addition: 0.0 67.53108511317987
hereu 2 67.53108511317987 67.53108511317987
check addition: 0.0 64.95666911121225
heredn 2 64.95666911121225 64.95666911121225
check addition: 0.0 66.8288484022487
hereu 3 66.8288484022487 66.8288484022487
check addition: 0.0 65.03191283799242
heredn 3 65.03191283799242 65.03191283799242
check addition: 0.0 65.59685421502218
hereu 4 65.59685421502218 65.59685421502218
check addition: 0.0 76.04664930782747
heredn 4 76.04664930782747 76.04664930782747


  0%|          | 0/1 [00:00<?, ?it/s]

check addition: 0.0 81.22555503086187
hereu 0 81.22555503086187 81.22555503086187
check addition: 0.0 105.10863906866871
heredn 0 105.10863906866871 105.10863906866871
check addition: 0.0 89.91790205310099
hereu 1 89.91790205310099 89.91790205310099
check addition: 0.0 94.55445853667334
heredn 1 94.55445853667334 94.55445853667334
check addition: 0.0 93.9780026669614
hereu 2 93.9780026669614 93.9780026669614
check addition: 0.0 90.34936218464281
heredn 2 90.34936218464281 90.34936218464281
check addition: 0.0 92.978776529897
hereu 3 92.978776529897 92.978776529897
check addition: 0.0 90.47451075271238
heredn 3 90.47451075271238 90.47451075271238
check addition: 0.0 93.38853493111674
hereu 4 93.38853493111674 93.38853493111674
check addition: 0.0 87.05710792518221
heredn 4 87.05710792518221 87.05710792518221
u 6131.048818707466
dn 7942.0457082390785
u 6795.103072792292
dn 7136.572320759296
u 7093.402016967535
dn 6828.47068926692
u 7022.963013857603
dn 6833.176007270813
u 7046.3549752235

'\nh_gg_vbf =[]\nfor tcomp in ltargetcomp: \n       FillHist("gg",tcomp,1,h_gg_vbf)\n\nh_gg_vh =[]\nfor tcomp in ltargetcomp: \n       FillHist("gg",tcomp,2,h_gg_vh)     \n        \n \n       \n#=========================================  \nh_ew_unt =[]\nfor tcomp in ltargetcomp: \n       FillHist("VBF",tcomp,0,h_ew_unt)\n\nh_ew_vbf =[]\nfor tcomp in ltargetcomp: \n       FillHist("VBF",tcomp,1,h_ew_vbf)\n\nh_ew_vh =[]\nfor tcomp in ltargetcomp: \n       FillHist("VBF",tcomp,2,h_ew_vh)      \n              \n#=========================================  \nh_bkg_unt =[]\nFillHist("ZZTo4l",0,0,h_bkg_unt)\n\nh_bkg_vbf =[]\nFillHist("ZZTo4l",0,1,h_bkg_vbf)\n    \nh_bkg_vh =[]\nFillHist("ZZTo4l",0,2,h_bkg_vh)      \n'

Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hf_temp (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ht_tempdn (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ht_tempdn (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ht_tempdn (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ht_tempdn (Potential memory leak).
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
Warning in <TFile::Append>: Replacing existing TH1: htt_tempu (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: htt_tempd (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: htt_tempu (Pot

In [68]:
c1 = ROOT.TCanvas("c1","c1",1800,1200)
c1.Divide(3,2)


leg = ROOT.TLegend(.5,.5,.90,.93)
leg.SetBorderSize(0)
leg.SetFillColor(0)
leg.SetFillStyle(0)
leg.SetTextFont(42)
leg.SetTextSize(0.035)



print (len(h_gg_unt))
for i, ht in enumerate(h_gg_unt): 
    print (i,ht.Integral())
    
    

print (len(h_gg_unt))
for i in range(0,len(h_gg_unt)): 
    h_gg_unt[i].SetLineStyle(2)
h_gg_unt[10].SetLineStyle(1)
h_gg_unt[10].SetLineColor(2)
h_gg_unt[2].SetLineColor(4)
h_gg_unt[3].SetLineColor(4)

c1.cd(1).SetLogx()

#h_gg_unt[10].Draw()

h_ratio1u = copy.deepcopy(h_gg_unt[10])
h_ratio1u.Reset()
h_ratio1d = copy.deepcopy(h_ratio1u)

line1 = ROOT.TLine(0,1.0,12000,1.0)
line1.SetLineStyle(2)
line1.SetLineColor(3)


h_gg_unt[10].GetXaxis().SetTitle("m4l  [GeV]")

print(h_gg_unt[0].Integral(),h_gg_unt[2].Integral(),h_gg_unt[3].Integral())
#h_gg_unt[0].Draw("same")
#h_gg_unt[1].Draw("same")

h_ratio1u.Divide(h_gg_unt[0],h_gg_unt[10])
h_ratio1d.Divide(h_gg_unt[1],h_gg_unt[10])
h_ratio1u.SetLineColor(1)
h_ratio1u.SetMaximum(1.3)
h_ratio1u.SetMinimum(0.8)
h_ratio1u.Draw("hist")
h_ratio1d.Draw("hist,same")
h_ratio1d.SetName("ratio_qcdrenom_dn")
h_ratio1u.SetName("ratio_qcdrenom_up")

leg.Clear()
leg.SetHeader("QCDscale Renormalization ")
leg.Draw()
line1.Draw()
c1.cd(2).SetLogx()


h_ratio2u = copy.deepcopy(h_gg_unt[10])
h_ratio2u.Reset()
h_ratio2d = copy.deepcopy(h_ratio2u)

h_ratio2u.Divide(h_gg_unt[2],h_gg_unt[10])
h_ratio2d.Divide(h_gg_unt[3],h_gg_unt[10])
h_ratio2u.SetLineColor(1)

h_ratio2u.Draw("hist")
h_ratio2d.Draw("hist,same")


h_ratio2d.SetName("ratio_qcdfact_dn")
h_ratio2u.SetName("ratio_qcdfact_up")



h_ratio2u.SetMaximum(1.3)
h_ratio2u.SetMinimum(0.8)
#h_gg_unt[10].Draw()
print(h_gg_unt[0].Integral(),h_gg_unt[2].Integral(),h_gg_unt[3].Integral())
#h_gg_unt[2].Draw("same")
#h_gg_unt[3].Draw("same")



leg2 = copy.deepcopy(leg)
leg2.Clear()
leg2.SetHeader("QCDscale Factorization ")
leg2.Draw()
line1.Draw()

c1.cd(3).SetLogx()
#h_gg_unt[10].Draw()
print(h_gg_unt[0].Integral(),h_gg_unt[2].Integral(),h_gg_unt[3].Integral())
#h_gg_unt[4].Draw("same")
#h_gg_unt[5].Draw("same")

h_ratio3u = copy.deepcopy(h_gg_unt[10])
h_ratio3u.Reset()
h_ratio3d = copy.deepcopy(h_ratio3u)

h_ratio3u.Divide(h_gg_unt[4],h_gg_unt[10])
h_ratio3d.Divide(h_gg_unt[5],h_gg_unt[10])
h_ratio3u.SetLineColor(1)
h_ratio3u.SetMaximum(1.3)
h_ratio3u.SetMinimum(0.8)
h_ratio3u.Draw("hist")
h_ratio3d.Draw("hist,same")

h_ratio3d.SetName("ratio_pdf_dn")
h_ratio3u.SetName("ratio_pdf_up")


leg3 = copy.deepcopy(leg)
leg3.Clear()
leg3.SetHeader("PDF variation")
leg3.Draw()
line1.Draw()

c1.cd(4).SetLogx()
#h_gg_unt[10].Draw()
print(h_gg_unt[0].Integral(),h_gg_unt[2].Integral(),h_gg_unt[3].Integral())
#h_gg_unt[6].Draw("same")
#h_gg_unt[7].Draw("same")


h_ratio4u = copy.deepcopy(h_gg_unt[10])
h_ratio4u.Reset()
h_ratio4d = copy.deepcopy(h_ratio4u)

h_ratio4u.Divide(h_gg_unt[6],h_gg_unt[10])
h_ratio4d.Divide(h_gg_unt[7],h_gg_unt[10])
h_ratio4u.SetLineColor(1)

h_ratio4u.SetMaximum(1.3)
h_ratio4u.SetMinimum(0.8)

h_ratio4u.Draw("hist")
h_ratio4d.Draw("hist,same")

h_ratio4d.SetName("ratio_astrong_dn")
h_ratio4u.SetName("ratio_astrong_up")






leg4 = copy.deepcopy(leg)
leg4.Clear()
leg4.SetHeader("alpha strong variation")
leg4.Draw()
line1.Draw()


c1.cd(5).SetLogx()
#h_gg_unt[10].Draw()
print(h_gg_unt[0].Integral(),h_gg_unt[2].Integral(),h_gg_unt[3].Integral())
#h_gg_unt[8].Draw("same")
#h_gg_unt[9].Draw("same")


h_ratio5u = copy.deepcopy(h_gg_unt[10])
h_ratio5u.Reset()
h_ratio5d = copy.deepcopy(h_ratio5u)

h_ratio5u.Divide(h_gg_unt[8],h_gg_unt[10])
h_ratio5d.Divide(h_gg_unt[9],h_gg_unt[10])
h_ratio5u.SetLineColor(1)
h_ratio5u.SetMaximum(1.5)
h_ratio5u.SetMinimum(0.5)
h_ratio5u.Draw("hist")
h_ratio5d.Draw("hist,same")
h_ratio5d.SetName("ratio_pythiascale_dn")
h_ratio5u.SetName("ratio_pythiascale_up")

line1.Draw()




leg5 = copy.deepcopy(leg)
leg5.Clear()
leg5.SetHeader("Pythia scale variation")
leg5.Draw()
#h_gg_unt[2].Draw("same")
#h_gg_unt[3].Draw("same")

c1.Draw()

c1.SaveAs("Theory_gghsignal_uncertainties_ratio.pdf")
c1.SaveAs("Theory_gghsignal_uncertainties_ratio.png")



fout = ROOT.TFile("LHEsyst_ratio.root","recreate")
fout.cd()
for i,htt in enumerate(h_gg_unt): 
   htt.SetName("htest") 
   htt.Write()

fout.Close()


fout1 = ROOT.TFile("Theory_gg_syst_ratio.root","recreate")
fout1.cd()

h_ratio1d.Write()
h_ratio1u.Write()
h_ratio2d.Write()
h_ratio2u.Write()
h_ratio3d.Write()
h_ratio3u.Write()
h_ratio4d.Write()
h_ratio4u.Write()
h_ratio5d.Write()
h_ratio5u.Write()



fout1.Close()



11
0 6131.048818707466
1 7942.0457082390785
2 6795.103072792292
3 7136.572320759296
4 7093.402016967535
5 6828.47068926692
6 7022.963013857603
7 6833.176007270813
8 7046.354975223541
9 9468.112914383411
10 6954.194003462791
11
6131.048818707466 6795.103072792292 7136.572320759296
6131.048818707466 6795.103072792292 7136.572320759296
6131.048818707466 6795.103072792292 7136.572320759296
6131.048818707466 6795.103072792292 7136.572320759296
6131.048818707466 6795.103072792292 7136.572320759296


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Info in <TCanvas::Print>: pdf file Theory_gghsignal_uncertainties_ratio.pdf has been created
Info in <TCanvas::Print>: png file Theory_gghsignal_uncertainties_ratio.png has been created


### Add to 'hlist' if the output looks reasonable above

In [ ]:
c1 = ROOT.TCanvas("c1","c1",1800,600)
c1.Divide(3,1)
h_bkg_unt[3].SetLineColor(2)
h_bkg_vh[3].SetLineColor(2)
h_bkg_vbf[3].SetLineColor(2)

h_bkg_unt[1].SetLineStyle(2)
h_bkg_vh[1].SetLineStyle(2)
h_bkg_vbf[1].SetLineStyle(2)


h_bkg_unt[2].SetLineStyle(2)
h_bkg_vh[2].SetLineStyle(2)
h_bkg_vbf[2].SetLineStyle(2)


c1.cd(1).SetLogy()
#c1.cd(1)

print (h_bkg_unt[1].Integral())
h_bkg_unt[3].Draw("hist")
h_bkg_unt[3].GetXaxis().SetTitle("qqZZ Untagged background m4l [GeV]")
h_bkg_unt[1].Draw("hist,same")
h_bkg_unt[2].Draw("hist,same")
c1.cd(2).SetLogy()
#c1.cd(2)
h_bkg_vh[3].Draw("hist")
h_bkg_vh[3].GetXaxis().SetTitle("qqZZ VHtagged background m4l [GeV]")
h_bkg_vh[1].Draw("hist,same")
h_bkg_vh[2].Draw("hist,same")



c1.cd(3).SetLogy()
#c1.cd(3)
h_bkg_vbf[3].Draw("hist")
h_bkg_vbf[3].GetXaxis().SetTitle("qqZZ VBFtagged background m4l [GeV]")
h_bkg_vbf[1].Draw("hist,same")
h_bkg_vbf[2].Draw("hist,same")


ROOT.gStyle.SetOptStat(1)
c1.Draw()
ROOT.gStyle.SetOptStat(1)
c1.Update()

c1.SaveAs("back_ew_qqzz.png")
c1.SaveAs("back_ew_qqzz.pdf")


In [ ]:
print  (h_gg_unt)

In [ ]:
print (h_list)

In [ ]:
hlist

In [ ]:
for h in h_list:
    print (h)
    print(h.Integral())

# Unroll and Visualize all histograms in 'hlist'

##### Do not proceed if your 'hlist' is not filled. Recall it expects 7 histograms ordered like: ['ggH SIG', 'ggH BSI', 'ggH BKG', 'EW SIG', 'EW BSI', 'EW BKG', 'qqbar BKG']
##### If you simply want to test the unrolling section, just fill the list with 7 copies of the same TH3F histogram

### Unroll all 7 histograms

In [ ]:
hlist1d = []
hlisttemp = h_list
for hf in hlisttemp:
    totgbins = hf.GetNbinsX() * hf.GetNbinsY() * hf.GetNbinsZ()
    totgedges = np.arange(totgbins+1, dtype='float64')

    #hf1d = ROOT.TH1F("hf1d","",len(totgedges)-1, totgedges)
    hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
    hf1d.SetDirectory(0)

    bincount = 1

    for xbin in trange(hf.GetNbinsX()):
        for ybin in range(hf.GetNbinsY()):
            for zbin in range(hf.GetNbinsZ()):
                gbin = hf.GetBin(xbin+1, ybin+1, zbin+1)

                #if gbin<400: print(xbin, ybin, zbin)
                #print(gbin)
                #print(bincount)
                if bincount>totgbins: 
                    print("WHAT")
                    break
                    
                #if hf.GetBinContent(gbin) != 0: print(hf.GetBinContent(gbin))
                #print(hf.GetBinContent(gbin))
                
                hf1d.SetBinContent(bincount, hf.GetBinContent(gbin))
                bincount += 1
                #if hf.GetBinContent(gbin) != 0: print("YAY")
    
    hlist1d.append(hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

### Isolate INT from the BSI histograms and also add the total combination of statistics as a new histogram at the end

In [ ]:
hlist1d[1].Add(hlist1d[0], -1)
hlist1d[1].Add(hlist1d[2], -1)

hlist1d[4].Add(hlist1d[3], -1)
hlist1d[4].Add(hlist1d[5], -1)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
hf1d.SetDirectory(0)
hf1d.Add(hlist1d[1])
hlist1d.insert(1, hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
hf1d.SetDirectory(0)
hf1d.Add(hlist1d[5])
hlist1d.insert(5, hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
for i in range(hlist1d[1].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[1].GetBinContent(b) >= 0:
        hlist1d[1].SetBinContent(b, 0)

for i in range(hlist1d[2].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[2].GetBinContent(b) <= 0:
        hlist1d[2].SetBinContent(b, 0)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
for i in range(hlist1d[5].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[5].GetBinContent(b) >= 0:
        hlist1d[5].SetBinContent(b, 0)

for i in range(hlist1d[6].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[6].GetBinContent(b) <= 0:
        hlist1d[6].SetBinContent(b, 0)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
for i in range(len(hlist1d)):
    hf1d.Add(hlist1d[i])
hlist1d.append(hf1d)
print(hf1d.Integral())

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hlist1d[1].Scale(-1)
hlist1d[5].Scale(-1)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1d = ROOT.TH1F("hf1d","",totgbins, 0, totgbins)
hf1d.Add(hlist1d[-1])

In [ ]:
hlist1d.append(hf1d)

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

In [ ]:
hf1dt = hlist1d.pop(-1)

In [ ]:
for i in range(hlist1d[-1].GetXaxis().GetNbins()):
    b = i+1
    if hlist1d[-1].GetBinContent(b) <= 0:
        hlist1d[-1].SetBinContent(b, 0)

### Check the final template collection and yields for the datacard

In [ ]:
c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(hlist1d),1)
for i in range(len(hlist1d)):
    c1.cd(i+1)
    print(hlist1d[i].Integral())
    hlist1d[i].Draw("hist")

ROOT.gStyle.SetOptStat(1)
c1.Draw()

# Saving the whole thing to a .root file for use with a corresponding datacard

### Checking the names used to save the histograms

In [ ]:
for i in proc:
    print(i)

### Set directory and output file names

In [ ]:
diname = "/eos/user/s/skyriaco/SWAN_projects/Offshel_tempmaker/Output/"
foname = "Cat_4mu_137p1invfb_ggH4landVBF4landqqbar_mostEW_RECOpassedevents_mvar21d1reg20d2reg20.input.root"

In [ ]:
print(diname+foname)

### Write to a .root file

In [ ]:
fo = ROOT.TFile(diname+foname,"CREATE")

for i in range(len(proc)):
    hlist1d[i].Write(proc[i], ROOT.TFile.kSingleKey)

fo.Close()

### Check the saved file

In [ ]:
fo = ROOT.TFile(diname+foname)
# fo = ROOT.TFile("/eos/user/l/lkang/Active_Research/Templates/SManalysis_split/2017/Untag/4mu/Untag_4mu_137p1invfb_ggH4landVBF4landqqbar_mostEW_RECOpassedevents_mvar21d1reg20d2reg20.input.root")

fo.ls()

c1 = ROOT.TCanvas("c1","c1",4480,600)
c1.Divide(len(proc),1)
for i in range(len(proc)):
    c1.cd(i+1)
    eval("print(proc[i], '\t\t=', fo."+proc[i]+".Integral())")
#     eval("print(fo."+proc[i]+".Integral())")
    eval("fo."+proc[i]+".Draw('hist')")
print()    

c1.Draw()
fo.Close()